Configuração inicial
--
Começo importando a biblioteca [Python Jira](https://jira.readthedocs.io/), a qual me permite fazer uso da API REST do Jira.

Em seguida crio um objeto JIRA, passando a autenticação básica (``username`` e ``password``) e o servidor onde está o Jira Software.

In [223]:
from jira import JIRA
jira = JIRA(basic_auth=('xb218744', 'RFbAHDs3dcJ2e2z'), options={'server': 'http://localhost:8080'})

Explorando o objeto ``jira``
--
Aqui, verifico os tipos de issues que estão configurados.

In [224]:
issue_types = [{'id': issue_type.id, 'name': issue_type.name} for issue_type in jira.issue_types()]
issue_types

[{'id': '10003', 'name': 'Tarefa'},
 {'id': '10004', 'name': 'Sub-tarefa'},
 {'id': '10002', 'name': 'História'},
 {'id': '10005', 'name': 'Problema'},
 {'id': '10001', 'name': 'Épico'},
 {'id': '10000', 'name': 'Sub-task'}]

E aqui, exploro os status que podem ser atribuídos às issues.

In [225]:
statuses_types = [{'id': status_type.id, 'name': status_type.name} for status_type in jira.statuses()]
statuses_types

[{'id': '3', 'name': 'In Progress'},
 {'id': '10000', 'name': 'To Do'},
 {'id': '10001', 'name': 'Done'}]

In [226]:
d = [st for st in statuses_types if st.get('name') == 'To Do'][0].get('name')
d

'To Do'

Buscando os recursos necessários
--
Aqui, utilizo JQL (acrônimo para Jira Query Language) para definir alguns filtros para a busca por issues.

Defino o projeto como sendo ``PPJ``, além de que o tipo da issue deva estar entre os tipos padrões de issues (o que não inclui sub-task) e, por fim, que o tipo da issue não seja ``Épico`` (no caso utilizo o id 10001, para garantir que não haja problema com o idioma).

Após definir a variável ``jql``, crio uma variável chamada ``issues`` a qual armazena o resultado da função ``jira.search_issues()``, onde foi definida com os parâmetros ``maxResults=100``, para a quantidade máxima de resultados e o parâmetro ``expand='changelog'``, que expande a busca além das issues, retornando também o changelog de cada uma.

Por último, imprimo a lista de issues retornadas.

In [227]:
jql = 'project=PPJ AND issuetype in standardIssueTypes() AND issueType != 10001'
issues = jira.search_issues(jql, maxResults=100, expand='changelog', fields='summary, project')
issues

[<JIRA Issue: key='PPJ-13', id='10012'>,
 <JIRA Issue: key='PPJ-12', id='10011'>,
 <JIRA Issue: key='PPJ-11', id='10010'>,
 <JIRA Issue: key='PPJ-10', id='10009'>,
 <JIRA Issue: key='PPJ-9', id='10008'>,
 <JIRA Issue: key='PPJ-8', id='10007'>,
 <JIRA Issue: key='PPJ-7', id='10006'>,
 <JIRA Issue: key='PPJ-6', id='10005'>,
 <JIRA Issue: key='PPJ-5', id='10004'>,
 <JIRA Issue: key='PPJ-4', id='10003'>,
 <JIRA Issue: key='PPJ-3', id='10002'>,
 <JIRA Issue: key='PPJ-2', id='10001'>,
 <JIRA Issue: key='PPJ-1', id='10000'>]

Lista de issues
--
Aqui crio uma lista contendo as issues do projeto que atendem os critérios acima.

Para cada issue eu itero a lista de changelog ``issue.changelog.histories``, em seguida para cada histórico eu acesso os seus itens e também os itero, pois é nesse recurso que conseguimos o status da issue.

Após passar pela condição ``item.field == 'status'``, adiciono um dictionary para cada issue à lista criada na primeira linha da célula seguinte. 

In [297]:
raw_issues = []
for issue in issues:
    for history in issue.changelog.histories:
        for item in history.items:
            if item.field == 'status':
                transition_date = history.created
                raw_issues.append({
                    'id_project': issue.fields.project.key,
                    'id_issue': issue.key,
                    'summary': issue.fields.summary,
                    'to_do': transition_date if item.toString == 'To Do' else '',
                    'in_progress': transition_date if item.toString == 'In Progress' else '',
                    'done': transition_date if item.toString == 'Done' else '',
                    'from_status': item.fromString,
                    'to_status': item.toString
                })

Pandas e o DataFrame
--
Primeiramente importo a biblioteca pandas, a qual é utilizada para estruturar dados e aplicar ferramentas analíticas à eles. Mas neste caso, utilizo apenas para criar a estrutura do que será exportado para um arquivo CSV.

Crio o DataFrame a partir do dictionary criado anteriormente, defino o índice como sendo o campo ``id_issue`` e ordeno a lista baseado no ``to_status`` e seleciono as colunas que irão aparecer e em que ordem (neste caso é mais pela ordem do que pela seleção de fato, já que o dictionary já foi criado na estrutura esperada).

In [298]:
import pandas as pd
df = pd.DataFrame.from_dict(raw_issues).set_index(['id_issue']).sort_values(['to_status'])\
    [['summary', 'to_do', 'in_progress', 'done', 'from_status', 'to_status']]
df

,summary,to_do,in_progress,done,from_status,to_status
id_issue,,,,,,
PPJ-9,Tarefa de desenvolvimento 008,,,2019-01-30T15:57:06.522-0200,In Progress,Done
PPJ-4,Tarefa de desenvolvimento 003,,,2019-01-30T17:46:59.403-0200,To Do,Done
PPJ-9,Tarefa de desenvolvimento 008,,,2019-01-30T12:48:22.863-0200,To Do,Done
PPJ-10,Tarefa de desenvolvimento 009,,2019-01-30T17:46:52.892-0200,,To Do,In Progress
PPJ-9,Tarefa de desenvolvimento 008,,2019-01-30T12:48:05.619-0200,,To Do,In Progress
PPJ-9,Tarefa de desenvolvimento 008,,2019-01-30T15:57:05.111-0200,,To Do,In Progress
PPJ-6,Tarefa de desenvolvimento 005,,2019-01-30T17:47:21.255-0200,,To Do,In Progress
PPJ-7,Tarefa de desenvolvimento 006,,2019-01-30T17:47:29.705-0200,,To Do,In Progress
PPJ-3,Tarefa de desenvolvimento 002,2019-01-30T17:47:14.011-0200,,,To Do,To Do


Exportando o arquivo
--
Após tudo isso, exporto o DataFrame para um arquivo CSV, o qual poderá ser utilizado em qualquer ferramenta que o suporte.

In [299]:
df.to_csv('jira.csv')